In [59]:
import torch as t
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader


import matplotlib.pyplot as plt

In [60]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform = ToTensor(),
    target_transform=None

)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform = ToTensor(),
    target_transform=None

)
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [61]:
BATCH_SIZE = 32

train_dataloader = DataLoader(dataset = train_data,
                              shuffle=False,
                              batch_size=BATCH_SIZE
                            )
test_dataloader = DataLoader(dataset=test_data,
                             shuffle=True,
                             batch_size=BATCH_SIZE)



In [62]:
train_features_batch,train_labels_batch = next(iter(train_dataloader))

In [63]:
x = train_features_batch[0]
print(f"Before Flattening:{x.shape}")
flattend = nn.Flatten()
z = flattend(x)
print(f"After Flattening:{z.shape}")

Before Flattening:torch.Size([1, 28, 28])
After Flattening:torch.Size([1, 784])


In [64]:
class FashionMNISTModelv0(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_nodes:int,
               output_shape:int):
    super().__init__()
    self.layer_Stack = nn.Sequential(nn.Flatten(),
                                     nn.Linear(in_features=input_shape,out_features=hidden_nodes),
                                     nn.Linear(in_features=hidden_nodes,out_features=output_shape))

  def forward(self,x:t.Tensor)-> t.Tensor:
    return self.layer_Stack(x)






In [65]:
t.manual_seed(42)
model = FashionMNISTModelv0(input_shape=784,#shape of the Input
                            hidden_nodes=30,
                            output_shape=len(class_names)
                            )



In [66]:
loss_fn = nn.CrossEntropyLoss()
optimizer = t.optim.SGD(params = model.parameters(),lr=0.01)

In [67]:
from tqdm.auto import tqdm
EPOCHS = 3

for epoch in tqdm(range(EPOCHS)):
  print(f"EPOCH:{epoch}\n=====")
  train_loss = 0
  for batch, (x,y) in enumerate(train_dataloader):
    model.train()
    y_pred = model(x)
    loss = loss_fn(y_pred,y)
    train_loss += loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 1000 == 0:
      print(f"Current Batch:{batch*len(x)}/{len(train_dataloader.dataset)}")


  model.eval()
  test_loss = 0
  with t.inference_mode():
    for x_test,y_test in test_dataloader:
      test_pred = model(x_test)

      test_loss += loss_fn(test_pred,y_test)
  print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}")




  0%|          | 0/3 [00:00<?, ?it/s]

EPOCH:0
=====
Current Batch:0/60000
Current Batch:32000/60000

Train loss: 1630.81506 | Test loss: 195.20644
EPOCH:1
=====
Current Batch:0/60000
Current Batch:32000/60000


KeyboardInterrupt: 

In [68]:
from tqdm.auto import tqdm

# Set the seed and start the timer
t.manual_seed(42)

# Set the number of epochs (we'll keep this small for faster training times)
epochs = 3

# Create training and testing loop
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")
    ### Training
    train_loss = 0
    # Add a loop to loop through training batches
    for batch, (X, y) in enumerate(train_dataloader):
        model.train()
        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss (per batch)
        loss = loss_fn(y_pred, y)
        train_loss += loss # accumulatively add up the loss per epoch

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Print out how many samples have been seen
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

    # Divide total train loss by length of train dataloader (average loss per batch per epoch)
    train_loss /= len(train_dataloader)

    ### Testing
    # Setup variables for accumulatively adding up loss and accuracy
    test_loss, test_acc = 0, 0
    model.eval()
    with t.inference_mode():
        for X, y in test_dataloader:
            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss (accumatively)
            test_loss += loss_fn(test_pred, y) # accumulatively add up the loss per epoch

            # 3. Calculate accuracy (preds need to be same as y_true)
            print(f"EPOCH:{epoch}|Test_Loss:{test_loss}")

        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(test_dataloader)

        # Divide total accuracy by length of test dataloader (per batch)
        test_acc /= len(test_dataloader)

    ## Print out what's happening
    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")



  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
-------
Looked at 0/60000 samples
Looked at 12800/60000 samples
Looked at 25600/60000 samples
Looked at 38400/60000 samples
Looked at 51200/60000 samples
EPOCH:0|Test_Loss:0.34297212958335876
EPOCH:0|Test_Loss:0.7608821392059326
EPOCH:0|Test_Loss:1.2886879444122314
EPOCH:0|Test_Loss:1.6235324144363403
EPOCH:0|Test_Loss:2.245941638946533
EPOCH:0|Test_Loss:2.6552906036376953
EPOCH:0|Test_Loss:3.060133934020996
EPOCH:0|Test_Loss:3.5930488109588623
EPOCH:0|Test_Loss:4.239079475402832
EPOCH:0|Test_Loss:4.660447597503662
EPOCH:0|Test_Loss:5.124104022979736
EPOCH:0|Test_Loss:5.680990695953369
EPOCH:0|Test_Loss:6.021175384521484
EPOCH:0|Test_Loss:6.603447914123535
EPOCH:0|Test_Loss:6.878861427307129
EPOCH:0|Test_Loss:7.341822147369385
EPOCH:0|Test_Loss:7.827346324920654
EPOCH:0|Test_Loss:8.203804016113281
EPOCH:0|Test_Loss:8.574365615844727
EPOCH:0|Test_Loss:9.12988567352295
EPOCH:0|Test_Loss:9.666790008544922
EPOCH:0|Test_Loss:10.233168601989746
EPOCH:0|Test_Loss:10.791192054748535
E

In [69]:
class FashionMnistv2(nn.Module):
  def __init__(self,input_shape:int,hidden_shape:int,output_shape:int):
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(input_shape,hidden_shape,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_shape,hidden_shape,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_shape,hidden_shape,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_shape,hidden_shape,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(hidden_shape*0,output_shape)

    )
    def forward(self,x:t.Tensor):
      x = self.conv_block_1 = nn.Sequential(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x


In [70]:
t.manual_seed(42)
model2 = FashionMnistv2(1,100,len(class_names))

loss_fn = nn.CrossEntropyLoss()
optimizer = t.optim.SGD(params=model2.parameters(),lr=0.001)


In [ ]:
from tqdm.auto import tqdm
EPOCHS = 5

for epoch in tqdm(range(EPOCHS)):
  print(f"EPOCH:{epoch}\n=====")
  train_loss = 0
  for batch, (x,y) in enumerate(train_dataloader):
    model2.train()
    y_pred = model(x)
    loss = loss_fn(y_pred,y)
    train_loss += loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 1000 == 0:
      print(f"Current Batch:{batch*len(x)}/{len(train_dataloader.dataset)}")


  model2.eval()
  test_loss = 0
  with t.inference_mode():
    for x_test,y_test in test_dataloader:
      test_pred = model(x_test)

      test_loss += loss_fn(test_pred,y_test)
  print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}")




  0%|          | 0/5 [00:00<?, ?it/s]

EPOCH:0
=====
Current Batch:0/60000
Current Batch:32000/60000

Train loss: 831.62866 | Test loss: 149.43117
EPOCH:1
=====
Current Batch:0/60000
Current Batch:32000/60000

Train loss: 831.62866 | Test loss: 149.45775
EPOCH:2
=====
Current Batch:0/60000


In [ ]:

def make_prodic(model:t.nn.Module,data:list):
  pred_probs = []
  model.eval()
  with t.inference_mode():
    for sample in data:
      sample = t.unsqueeze(sample,dim=0)

      pred_logt = model(sample)
      pred_prob = t.softmax(pred_logt.squeeze(),dim=0)
      pred_probs.append(pred_prob)
    return t.stack(pred_prob)


In [ ]:
import random
random.seed(42)
test_samples = []
test_labels=[]

for sample,label in random.sample(list(test_data),k=9):
  test_samples.append(sample)
  test_labels.append(label)


plt.imshow(test_samples[0].squeeze(),cmap="gray")

In [ ]:
a = make_prodic(model=model2,data=test_samples)